#Telecom Domain Read & Write Ops Hackathon - Building Datalake & Lakehouse
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

##2. Filesystem operations
1. Write dbutils.fs code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

##3. Spark Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

## Spark Write Operations using 
- csv, json, orc, parquet, delta, saveAsTable, insertInto, xml with different write mode, header and sep options

##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1.Write customer data into CSV format using overwrite mode
customer_df = spark.read.csv(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer",
    header=True,
    inferSchema=True
)

customer_df.write.csv(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer",
    mode="overwrite",
    header=True
)



In [0]:
#2.Write usage data into CSV format using append mode

usage_df=spark.read.csv("/Volumes/workspace/default/usage_metrics/mobile_os_usage.csv",header=True,inferSchema=True)
usage_df.write.csv("/Volumes/workspace/default/usage_metrics",mode="append",header=True)
    

In [0]:
#3.Write tower data into CSV format with header enabled and custom separator (|)

tower_df =spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv",header=True,inferSchema=True)
tower_df.write.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/output", mode="overwrite", header=True, sep="|")

In [0]:
#4. Read the tower data in a dataframe and show only 5 rows
tower_df =spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/output",header=True,inferSchema=True)
tower_df.show(5)

In [0]:
#5.Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++

dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/output')


#downloaded and opened in notepad 

#output file
"event_id|customer_id|tower_id|signal_strength|timestamp"
"5001|101|TWR01|-80|2025-01-10 10:21:54"
"5004|104|TWR05|-75|2025-01-10 11:01:12"



##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1. write customer data into json using overwrite mode
customer_df = spark.read.csv(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer",
    header=True)
    
customer_df.write.json(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/output",
    mode="overwrite")


In [0]:
#2. Write usage data into JSON format using append mode and snappy compression format
usage_df=spark.read.csv("/Volumes/workspace/default/usage_metrics/mobile_os_usage.csv",header=True,inferSchema=True)
usage_df.write.json("/Volumes/workspace/default/usage_metrics/output",mode="append",compression="snappy")

In [0]:
# Remove the output directory before writing (run in a notebook cell, not Python)
dbutils.fs.rm(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/output",
    True
)


#3.Write tower data into JSON format using ignore mode and observe the behavior of this mode
tower_df =spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv",header=True,inferSchema=True)
tower_df.write.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/output", mode="ignore")


#4.Read the tower data in a dataframe and show only 5 rows.

tower_df =spark.read.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/output")
display(tower_df.limit(5))

##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1.Write customer data into Parquet format using overwrite mode and in a gzip format
customer_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer", header=True,inferSchema=True)
customer_df.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/output", mode="overwrite",compression="gzip")


# 2. Write usage data into Parquet format using overwrite mode and snappy compression
usage_df = spark.read.csv(
    "/Volumes/workspace/default/usage_metrics/mobile_os_usage.csv",
    header=True,
    inferSchema=True
)
usage_df.write.parquet(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/output_parquet",
    mode="overwrite",
    compression="snappy"
)

#3.Write tower data into Parquet format with gzip compression option

tower_df =spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv",header=True,inferSchema=True)
tower_df.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/output", mode="overwrite",compression="gzip")

#4.Read the usage data in a dataframe and show only 5 rows.

usage_df=spark.read.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/output_parquet")
display(usage_df.limit(5))

#5.Downloaded and got the file in notepad++



##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
# 1. Customer → ORC (overwrite)
customer_df = spark.read.csv(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer",
    header=True,
    inferSchema=True
)

customer_df.write \
    .mode("overwrite") \
    .orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer_orc")


# 2. Usage → ORC (append)
usage_df = spark.read.csv(
    "/Volumes/workspace/default/usage_metrics/mobile_os_usage.csv",
    header=True,
    inferSchema=True
)

usage_df.write \
    .mode("append") \
    .orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/output_orc")


# 3. Tower → ORC (overwrite)
tower_df = spark.read.csv(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv",
    header=True,
    inferSchema=True
)

tower_df.write \
    .mode("overwrite") \
    .orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/output")


# 4. Read usage ORC & show 5 rows
usage_read_df = spark.read.orc(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/output_orc"
)

display(usage_read_df.limit(5))


##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

In [0]:
1.#Write customer data into Delta format using overwrite mode
customer_df = spark.read.csv(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer",
    header=True,
    inferSchema=True)
customer_df.write.format("delta").mode("overwrite").save(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer_delta"
)
#2.Write usage data into Delta format using append mode
usage_df = spark.read.csv(
    "/Volumes/workspace/default/usage_metrics/mobile_os_usage.csv",
    header=True,
    inferSchema=True
)


#3.Write Tower data into Delta & observe structure
tower_df = spark.read.csv(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv",
    header=True,
    inferSchema=True)
tower_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_delta")

##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using saveAsTable() as a managed table
2. Write usage data using saveAsTable() with overwrite mode
3. Drop the managed table and verify data removal
4. Go and check the table overview and realize it is in delta format in the Catalog.
5. Use spark.read.sql to write some simple queries on the above tables created.


In [0]:
#Write customer data using saveAsTable() as a managed table
#customer_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer",header=True,inferSchema=True)
#customer_df.write.format("delta").mode("overwrite").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer_delta")
#customer_df.write.format("delta").saveAsTable("default.customer_managed")

 # Write usage data using saveAsTable() with overwrite mode
#usage_df = spark.read.csv("/Volumes/workspace/default/usage_metrics/mobile_os_usage.csv",header=True,inferSchema=True)
#usage_df.write.format("delta").mode("overwrite").saveAsTable("default.usage_managed")

#Drop the managed table and verify data removal
#spark.sql("DROP TABLE IF EXISTS default.customer_managed")
#spark.sql("DROP TABLE IF EXISTS default.usage_managed")

#Go and check the table overview and realize it is in delta format in the Catalog.
#spark.sql("DESCRIBE EXTENDED default.usage_managed")


#Use spark.read.sql to write some simple queries on the above tables created.

spark.sql("SELECT * FROM default.customer_managed").display()

##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using insertInto() in a new table and find the behavior
2. Write usage data using insertTable() with overwrite mode

In [0]:
#Write customer data using insertInto() in a new table and find the behavior
customer_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer",header=True,inferSchema=True)

customer_df.write.insertInto("default.customer_managed")

#Write usage data using insertTable() with overwrite mode
usage_df = spark.read.csv("/Volumes/workspace/default/usage_metrics/mobile_os_usage.csv",header=True,inferSchema=True)
usage_df.write.insertInto("default.usage_managed",mode="overwrite")


##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust
2. Write usage data into XML format using overwrite mode with the rowTag as usage
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

In [0]:
#spark.read.format("xml").option("rowTag", "test").load("/tmp/dummy")
com.databricks:spark-xml_2.13:0.16.0



##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml) 
1. Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.

(Small ➜ Big)

1️⃣ ORC
2️⃣ Parquet
3️⃣ Delta
4️⃣ JSON
5️⃣ CSV
6️⃣ XML

###15. Try to do permutation and combination of performing Schema Migration & Data Conversion operations like...
1. Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format
2. Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format
3. Read any one of the above delta data in a dataframe and write it to dbfs in a xml format
4. Read any one of the above delta table in a dataframe and write it to dbfs in a json format
5. Read any one of the above delta table in a dataframe and write it to another table

##16. Do a final exercise of defining one/two liner of... 
1. When to use/benifits csv
2. When to use/benifits json
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta
6. When to use/benifit xml
7. When to use/benifit delta tables


##CSV/JSON/XML → Data exchange
##Parquet/ORC → Analytics
##Delta/Delta Tables → Reliable, production Lakehouse